In [2]:
import torch
from torch import nn

def corr2d(X, K):
    # 卷积
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]]) - 1
Y = torch.tensor([[1, 2], [3, 4]]) - 1
print(corr2d(X, Y))

tensor([[19., 25.],
        [37., 43.]])


# 二维卷积层

In [25]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1)) # 用于生成一个从标准正态分布（均值为0，方差为1）中抽取的随机数张量。
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

# 图像中物体边缘检测

In [26]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
print(X.shape)
X

torch.Size([6, 8])


tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [27]:
K = torch.tensor([[1, -1]])

In [28]:
Y = corr2d(X, K)
print(Y.shape)
Y

torch.Size([6, 7])


tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

# 通过数据学习核数组

In [29]:
# 构造一个核数组形状是(1, 2)的二维卷积层
conv2d = Conv2D(kernel_size=(1, 2))

step = 20
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat - Y) ** 2).sum()
    l.backward()
    
    # 梯度下降
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad
    
    # 梯度清零
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    if (i + 1) % 5 == 0:
        print('Step %d, loss %.3f' % (i + 1, l.item()))

Step 5, loss 2.019
Step 10, loss 0.293
Step 15, loss 0.051
Step 20, loss 0.011


In [30]:
print("weight: ", conv2d.weight.data)  
print("bias: ", conv2d.bias.data)

weight:  tensor([[ 1.0304, -1.0170]])
bias:  tensor([-0.0075])
